In [11]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using System.Collections.Generic;
using System.IO;
using System;
using System.Text.RegularExpressions;

In [12]:
var fileExists = File.Exists("input.txt");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

This file exists.  Nothing has blown up.....yet.

In [13]:
string[] lines = System.IO.File.ReadAllLines("input.txt");
lines

[ Game 1: 4 blue, 7 red, 5 green; 3 blue, 4 red, 16 green; 3 red, 11 green, Game 2: 20 blue, 8 red, 1 green; 1 blue, 2 green, 8 red; 9 red, 4 green, 18 blue; 2 green, 7 red, 2 blue; 10 blue, 2 red, 5 green, Game 3: 2 red, 5 green, 1 blue; 3 blue, 5 green; 8 blue, 13 green, 2 red; 9 green, 3 blue; 12 green, 13 blue; 3 green, 3 blue, 1 red, Game 4: 1 red, 6 green, 4 blue; 3 green, 1 blue, 1 red; 7 blue, 1 red, 2 green, Game 5: 2 green, 9 blue, 1 red; 3 green, 1 blue, 3 red; 1 red, 4 blue, 9 green, Game 6: 2 blue, 5 red, 7 green; 5 blue, 8 red, 3 green; 2 red, 9 blue, 2 green, Game 7: 7 green, 7 blue, 2 red; 2 red, 7 green, 16 blue; 17 blue, 3 green, 3 red; 2 blue, 5 green, 3 red, Game 8: 4 red, 3 green; 9 green, 2 red, 2 blue; 1 red, 3 blue, 6 green, Game 9: 5 red, 3 green, 13 blue; 11 red, 15 blue, 1 green; 7 red, 2 blue, Game 10: 15 red, 3 green; 7 green, 4 blue, 11 red; 13 red, 13 blue; 2 blue, 5 green, 8 red, Game 11: 7 red, 3 green; 7 blue, 16 red, 4 green; 6 green, 6 blue, 12 red; 11 red, 4 green, 4 blue; 10 red, 6 blue, 2 green; 3 green, 7 red, 6 blue, Game 12: 1 blue, 2 red; 2 green, 15 blue; 6 green, 5 blue; 6 blue, 4 green; 5 blue, 3 green; 3 red, 3 blue, 10 green, Game 13: 10 red, 4 green; 9 red, 2 blue, 3 green; 6 red, 7 green, 1 blue; 9 red, 7 green, 1 blue; 3 blue; 3 blue, 3 red, 8 green, Game 14: 12 blue, 3 red, 4 green; 3 green, 1 red; 6 green, 16 blue, Game 15: 2 green, 3 red, 2 blue; 14 blue, 1 red, 17 green; 13 blue, 11 green, 10 red; 5 green, 7 red, 5 blue; 2 green, 3 blue, 6 red; 9 green, 2 blue, 5 red, Game 16: 2 blue, 1 red; 1 red, 2 green, 3 blue; 4 green, 9 blue, 3 red; 1 green, 4 red, 8 blue; 7 blue, 11 red, 1 green, Game 17: 9 green, 8 blue, 6 red; 8 red, 18 green, 1 blue; 18 red, 19 green, 1 blue, Game 18: 1 green, 4 red, 5 blue; 10 green, 8 blue; 12 green, 10 blue, Game 19: 3 red, 11 green, 12 blue; 16 green, 1 red, 20 blue; 9 green, 2 red, 14 blue; 5 blue, 2 green, 2 red; 20 blue, 3 red, 10 green; 4 green, 3 blue, Game 20: 17 red, 3 blue, 9 green; 6 green, 1 red, 7 blue; 6 red, 2 blue; 1 blue, 4 green, 5 red; 6 green, 5 red; 10 blue, 11 green, 2 red ... (80 more) ]

I'm going to make some functions this time, because some of the things need to be a bit more complex and replicable.

In [14]:
public static string UseMe()
{
    return "hey yo";
}

In [15]:
var asdfasdf = UseMe();
asdfasdf

hey yo

That works, so let's try to convert a line of game input into some form of object that shows the maximum cubes shown.

In [72]:
public class CubeBag
{
    // Properties to track the count of each color of cubes
    public int ID { get; set; }
    public int RedCubes { get; set; }
    public int GreenCubes { get; set; }
    public int BlueCubes { get; set; }

    // Method to display the current counts of each color of cubes
    public void DisplayCubeCounts()
    {
        Console.WriteLine("Red cubes: " + RedCubes);
        Console.WriteLine("Green cubes: " + GreenCubes);
        Console.WriteLine("Blue cubes: " + BlueCubes);
    }
}

public static CubeBag CalculateBag(string pulls)
{
    CubeBag bag = new CubeBag();
    //Console.Out.WriteLine("current line:" + pulls);
    string[] semicolonSplit = pulls.Split(";");

    var currentId = 0;

    List<string> allThePulls = new List<string>();

    foreach (string str in semicolonSplit)
    {
        //Console.Out.WriteLine("String portion:" + str);
        if (str.StartsWith("Game"))
        {
            string[] gameParts = str.Split(':');
            string gamePortion = gameParts[0];
            string numberPart = gamePortion.Substring(4);
            //Console.WriteLine("ID:" + numberPart);
            bool result = int.TryParse(numberPart, out currentId);
            //Console.Out.WriteLine("Game part: " + gamePortion);
            string[] scoreParts = gameParts[1].Split(','); 
            foreach (string p in scoreParts)
            {
                //Console.Out.WriteLine("Score Part: " + p);
                allThePulls.Add(p);
                (string color, int count) currentPull = CubeCounter(p);
                //Console.Out.WriteLine("Color:" + currentPull.color + " Count: " + currentPull.count);
            }

            
        }
        else
        {
            foreach (var part in str.Split(','))
            {
                //Console.Out.WriteLine("game part:" + part);
                allThePulls.Add(part);
            }   
        }

        //Console.Out.WriteLine("Here's all the parts:");
        foreach(string asdf in allThePulls)
        {
            //Console.Out.WriteLine("alltheParts part:" + asdf);
        }
    }

    // I have all the parts, now make the bag.
    // create default red, green, and blue counts.
    IDictionary<string, int> cubeValues = new Dictionary<string, int>();
    cubeValues.Add("green",0);
    cubeValues.Add("blue",0);
    cubeValues.Add("red",0);

    //Get all the new values, and only save the highest values.
    foreach(string pull in allThePulls)
    {
        (string color, int count) currentPull = CubeCounter(pull);
        //Console.Out.WriteLine("Color:" + currentPull.color + " Count: " + currentPull.count);
        int oldValue = cubeValues[currentPull.color];
        if(currentPull.count > oldValue)
        {
            cubeValues[currentPull.color] = currentPull.count;
        }
    }

    //translate dictionary to object
    bag.ID = currentId;
    bag.RedCubes = cubeValues["red"];
    bag.GreenCubes = cubeValues["green"];
    bag.BlueCubes = cubeValues["blue"];


    return bag;
}

public static (string, int) CubeCounter(string cubePull)
{
    string[] cubeParts = cubePull.Substring(1).Split(" ");
    string cubeColor = cubeParts[1];
    bool success = int.TryParse(cubeParts[0], out int cubeCount);
    if(!success)
    {
        cubeCount = 0;
    }


    return (cubeColor, cubeCount);
}



In [81]:
// I can create cubes, now I need to see if they are possible.
public static int AddBagIfPossible(CubeBag cb, int redCount = 12, int blueCount = 14, int greenCount = 13)
{
    if (redCount >= cb.RedCubes && blueCount >= cb.BlueCubes && greenCount >= cb.GreenCubes)
    {
        return cb.ID;
    }
    else
    {
        return 0;
    }
}

In [82]:
var bag0 = CalculateBag(lines[0]);

int sum = 0;

foreach(string line in lines)
{
    CubeBag bag = CalculateBag(line);
    int possibleId = AddBagIfPossible(bag);
    Console.WriteLine("Possible ID to add: " + possibleId);
    sum += possibleId;
}

//int val = AddBagIfPossible(bag0);

sum



Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 3
Possible ID to add: 4
Possible ID to add: 5
Possible ID to add: 6
Possible ID to add: 0
Possible ID to add: 8
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 13
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 16
Possible ID to add: 0
Possible ID to add: 18
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 25
Possible ID to add: 26
Possible ID to add: 27
Possible ID to add: 28
Possible ID to add: 0
Possible ID to add: 30
Possible ID to add: 0
Possible ID to add: 32
Possible ID to add: 33
Possible ID to add: 34
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 37
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 0
Possible ID to add: 41
Possible ID to add: 0
Possible ID to add: 43
Possible ID to add: 0
Possible ID to add

2512

That's good enough for the first star!

The next task wants me to multiply the minimum cube numbers for each color to each other and get the sum.

In [84]:
public static int CubePower(CubeBag cb)
{
    return cb.RedCubes * cb.BlueCubes * cb.GreenCubes;
}


int sum = 0;

foreach(string line in lines)
{
    CubeBag bag = CalculateBag(line);
    int powerNumber = CubePower(bag);
    Console.WriteLine("Cube ID: " + bag.ID + " Power: " + powerNumber);
    sum += powerNumber;
}


sum

Cube ID: 1 Power: 448
Cube ID: 2 Power: 900
Cube ID: 3 Power: 338
Cube ID: 4 Power: 42
Cube ID: 5 Power: 243
Cube ID: 6 Power: 504
Cube ID: 7 Power: 357
Cube ID: 8 Power: 108
Cube ID: 9 Power: 495
Cube ID: 10 Power: 1365
Cube ID: 11 Power: 672
Cube ID: 12 Power: 450
Cube ID: 13 Power: 240
Cube ID: 14 Power: 288
Cube ID: 15 Power: 2380
Cube ID: 16 Power: 396
Cube ID: 17 Power: 2736
Cube ID: 18 Power: 480
Cube ID: 19 Power: 960
Cube ID: 20 Power: 1870
Cube ID: 21 Power: 672
Cube ID: 22 Power: 512
Cube ID: 23 Power: 1008
Cube ID: 24 Power: 2184
Cube ID: 25 Power: 660
Cube ID: 26 Power: 336
Cube ID: 27 Power: 162
Cube ID: 28 Power: 2
Cube ID: 29 Power: 4680
Cube ID: 30 Power: 84
Cube ID: 31 Power: 252
Cube ID: 32 Power: 104
Cube ID: 33 Power: 616
Cube ID: 34 Power: 550
Cube ID: 35 Power: 972
Cube ID: 36 Power: 160
Cube ID: 37 Power: 520
Cube ID: 38 Power: 176
Cube ID: 39 Power: 576
Cube ID: 40 Power: 408
Cube ID: 41 Power: 468
Cube ID: 42 Power: 936
Cube ID: 43 Power: 40
Cube ID: 44 Power:

67335

And that's part 2!